In [10]:
import lmoments3 as lm
from lmoments3 import distr
import pandas as pd
import numpy as np
import os
from scipy.stats import kappa4
from scipy.special import gamma

In [11]:
results_folder = r'../example/results/discordancy'
if not os.path.exists(results_folder):
    os.mkdir(results_folder)

In [13]:

stations = ['Station1','Station2','Station3','Station4','Station5','Station6','Station7'] #Selection of stations of interest within a cluster
df_prec = pd.read_parquet('precipitations_daily_time_series.parquet')
lmoments_list_val = list()
u_list = list()
for sta, df_dia in df_prec.groupby('StationId'): 
    #Extracting data from db, clearing NANs and passing it into an array
    if(sta not in stations):continue
    df_station = df_dia[df_dia["StationId"]==sta]
    station = df_station[df_station['IR']>=0.2]['IR']
    array = station.to_numpy()
    x = array[~np.isnan(array)]
    n_x = len(x)
    sum_x = sum(x)
    #L-moment determination
    lamb1, lamb2, t3, t4 = lm.lmom_ratios(x, nmom=4)
    l_CV = lamb2/lamb1
    lmoments_list_val.append([sta, lamb1, lamb2, t3, t4, l_CV, n_x, sum_x])
    u_list.append([sta, l_CV, t3, t4, n_x])
df_lmoments = pd.DataFrame(lmoments_list_val, columns =['station', 'lamb1', 'lamb2', 't3', 't4', 'l_CV', 'n', 'sum_x'])
df_u = pd.DataFrame(u_list, columns =['station', 'l_CV', 't3', 't4', 'n'])
df_u.to_excel(os.path.join(results_folder, 'ui.xlsx'), index=False)

In [14]:
#DISCORDANCY
#ui average
sum_lCV = 0
sum_t3 = 0
sum_t4 = 0
N_ui = len(stations)
for i in range(0, N_ui):
    sum_lCV = sum_lCV + df_u.at[i,'l_CV']
    sum_t3 = sum_t3 + df_u.at[i,'t3']
    sum_t4 = sum_t4 + df_u.at[i,'t4']

u_mean = (sum_lCV/N_ui,sum_t3/N_ui,sum_t4/N_ui)

#A
A = np.zeros((3, 3))
for i in range(0, N_ui):
    list_ui = (df_u.at[i,'l_CV']-u_mean[0], df_u.at[i,'t3']-u_mean[1], df_u.at[i,'t4']-u_mean[2])
    ui_array = (np.array(list_ui)).reshape(1, -1)
    ui_array_T = (ui_array.T).reshape(-1, 1)
    A = A + np.dot(ui_array_T,ui_array)


# Calculate Discordancy of different sites i
Di = list()
for i in range(0, N_ui):
    list_ui = (df_u.at[i,'l_CV']-u_mean[0], df_u.at[i,'t3']-u_mean[1], df_u.at[i,'t4']-u_mean[2])
    ui_array = (np.array(list_ui)).reshape(1, -1)
    ui_array_T = (ui_array.T).reshape(-1, 1)
    D_i = N_ui/3*np.dot(ui_array,np.dot(np.linalg.inv(A),ui_array_T))
    Di.append([stations[i], np.mean(D_i)])
df_Di =pd.DataFrame(Di, columns=['station','Di'])
df_Di.to_excel(os.path.join(results_folder, 'Di.xlsx'), index=False)